In [ ]:
import sys
import os
# Append the library path to PYTHONPATH, so library can be imported.
sys.path.append(os.path.dirname(os.getcwd()))
import shutil
import datetime

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

from library import plot
from library import network as nw
from library import common as cm

In [ ]:
%run setup.py
%run Load_Clean_aux.py tune

seed = 666
np.random.seed(seed)

In [ ]:
if FEATURE_SET == 'normal_feature':
    ori_fea = ['M0', 'tau0_implvol0']
    sub_res = res_dir + 'Network/Normal_Feature/TuneHypers/'

if FEATURE_SET == 'delta_vega':
    ori_fea = ['delta_bs', '1_over_sqrt_tau', 'vega_n']
    sub_res = res_dir + 'Network/Delta_Vega/TuneHypers/'

if VIX:
    ori_fea += ['fake_vix']

os.makedirs(sub_res, exist_ok=True)

In [ ]:
df_val = df_train.loc[df_train['period0'] == 1].copy()
df_train = df_train.loc[df_train['period0'] == 0].copy()

##### Step 2: Standardize features
The training and validation set are standardized, and the resulting scaler shall be passed to standardize each Monte Carlo set later.

In [ ]:
use_fea = [x + '_t' for x in ori_fea] + ['cp_int']

scaler = StandardScaler().fit(X=df_train[ori_fea])
df_train, df_val = nw.standardize_feature([df_train, df_val], scaler, ori_fea)

In [ ]:
use_fea

##### Step 3: Specify a candidate set
We need to specify a set of candidate hyperparameters. Each candidate will be used to train a new network for a specified number of times.

In [ ]:
hypers = {
    'nodes_per_layer': (30, 30),
    'reg_alpha': 1e-4,
    'lr': 1e-4,
    'epochs': 300,
    'outact': 'linear'
}
lab_tune = 'reg_alpha'
value_set = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
num_run = 5 # 5

In [ ]:
sub_res

##### Step 4: Training step


In [ ]:
for value in value_set:
    # If you want to tune other hyper apart from alpha, 
    # change key name and directory name of the next two lines.
    hypers[lab_tune] = value
    alpha_dir = sub_res + f'para={value:.0e}/'
    
    alpha_dir_dict = {
        'ckp': alpha_dir + 'ckp/',
        'history': alpha_dir + 'history/',
        'metrics': alpha_dir + 'metrics/',
        'plot': alpha_dir + 'plot/'
    }
    for key, value in alpha_dir_dict.items():
        os.makedirs(value, exist_ok=True)
    
    # for each value, we train a (new) network multiple times.
    for i in range(num_run):   
        """
        Here, each checkpoint, history, and metrics corresponds to each run.
        """        
        # for each iteration, we record its performance on each of MC set.
        df_metrics = pd.DataFrame(index=range(NUM_TEST), columns=['BS', 'NN'])
        
        sub_res_paths = {
            'ckp': alpha_dir_dict['ckp'] + f'bestcp{i}.h5',
            'history': alpha_dir_dict['history'] + f'history{i}.csv',
            'plot': alpha_dir_dict['plot'] + f'losscurve{i}.png',
            'metrics': alpha_dir_dict['metrics'] + f'metrics{i}.csv'
        }
        history = nw.train_net_core(df_train, df_val, use_fea, hypers, sub_res_paths)
        nw.plot_history(history, sub_res_paths['plot'], df_train, df_val)
        
        """
        Each train is tested on many Monte Carlo sets.
        """
        for j in range(NUM_TEST):
            df_test = mc_sets[j].copy()
            [df_test] = nw.standardize_feature([df_test], scaler, ori_fea)
            
            delta_nn = nw.test_net_core(df_test, use_fea, sub_res_paths)
            
            """
            We calculate NN pnl and BS pnl.
            """
            pnl_nn = cm.calc_pnl(df_test, delta_nn)
            pnl_bs = cm.calc_pnl(df_test, df_test['delta_bs'])
            
            df_metrics.loc[j, 'NN'] = (pnl_nn**2).mean()
            df_metrics.loc[j, 'BS'] = (pnl_bs**2).mean()      
        
        df_metrics.to_csv(sub_res_paths['metrics'])

# Summarize tuning results
This section can be run independent of the above one, if the directory paths are given properly.

In [ ]:
num_run = 5
value_set = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]

In [ ]:
df_metrics_summary = pd.DataFrame(
    index=range(num_run), 
    columns=['BS'] + [f'para={v:.0e}' for v in value_set])

In [ ]:
for value in value_set:
    metrics_folder = sub_res + f'para={value:.0e}/metrics/'
    
    for i in range(num_run):
        df_metrics = pd.read_csv(metrics_folder + f'metrics{i}.csv', index_col=0)
        """
        'BS' errors do not change, whatever the alpha is.
        We take the average over all test sets for each run.
        """
        df_metrics_summary.loc[i, 'BS'] = df_metrics.mean()['BS'] 
        df_metrics_summary.loc[i, f'para={value:.0e}'] = df_metrics.mean()['NN']

df_metrics_summary = df_metrics_summary.astype('float32')

This summary table shows the average test metric over all the Monte Carlo data sets, for different hyperparamters and many runnings of network. 

In [ ]:
df_metrics_summary.to_csv(f'{sub_res}individual_mc_performance.csv')

In [ ]:
df_metrics_summary

In [ ]:
df_metrics_summary.describe().to_csv(f'{sub_res}summary_metrics.csv')

In [ ]:
df_metrics_summary.describe()

# Save results
Run this section, if you want to save all related checkpoint, history and plots permanently, so that they will not be covered.

In [ ]:
# Copy data setup file from the clean data folder, and then append network setup.
shutil.copy('setup.py', sub_res)

with open(f'{sub_res}tuning-setting.txt', 'w') as file:
    file.write('The following is TUNE setup.\n')
    file.write(f'Date and time = {datetime.datetime.now()}\n')
    for n, x in [
        ('Random seed', seed),
        ('Features used', use_fea),
        ('Hypers', hypers),
        ('Value set to tune from', value_set),
        ('Number of iterations', num_run)
    ]:
        file.write(f'{n} = {x}\n') 
    file.write(f'{lab_tune} is being tuned.')